In [2]:
import ollama
import pandas as pd
import json

from pdfminer.high_level import extract_text
import re
import pandas as pd
from nameparser import HumanName
from tqdm import tqdm




In [25]:


pdf_path = "../data/ERC-2024-AdG-panel-members.pdf"
pdf_text = extract_text(pdf_path)

# Definiere ein Beispiel-Output-Schema
schema_example = [{
        "Domain": "PE",
        "Subdomain": "Mathematics (PE1)",
        "Forename": "John",
        "Lastname": "Doe",
        "Fullname": "John Doe",
        "Type": "Chair",
        "ERC-Date": "2025",
        "review_panel": "PE1"
    },
    {
        "Domain": "LS",
        "Subdomain": "Life Sciences (LS2)",
        "Forename": "Jane",
        "Lastname": "Smith",
        "Fullname": "Jane Smith",
        "Type": "Member",
        "ERC-Date": "2025",
        "review_panel": "LS2"
    }
]

prompt = f"""
You are an expert text extractor and data structurer.

Your task:
Extract **all ERC panel members** from the provided text below.

### Extraction Rules
1. Each record must be represented as a **JSON object**.
2. The output must be a **single JSON array** (`[...]`) with one object per member.
3. Each object must include **exactly** these keys (use empty strings if information is missing):

{json.dumps(list(schema_example[0].keys()), ensure_ascii=False, indent=2)}

4. The field meanings:
   - "Domain": the ERC domain code (e.g., PE, LS, SH)
   - "Subdomain": the subdomain name (e.g., "Mathematics (PE1)")
   - "Forename": first name
   - "Lastname": last name
   - "Fullname": full name = Forename + Lastname
   - "Type": either "Chair" or "Member"
   - "ERC-Date": ERC-date, e.g., 2025 or 2024"
   - "review_panel": the panel code, e.g., PE1, LS2, SH3, etc.

5. If a person belongs to "Earth System Science (PE10)" or "Universe Sciences (PE9)",
   make sure they are assigned to the **correct** subdomain and not swapped.

6. Do **not** include explanations, markdown, or text outside the JSON list.
   The entire model output must be a valid JSON array.

---

### Text to extract from:
{pdf_text}

### Example Output
{json.dumps(schema_example, ensure_ascii=False, indent=2)}
"""

response = ollama.chat(
    model="llama3:instruct",
    messages=[{"role": "user", "content": prompt}],
    stream=True,
)

# Antwort puffern und gleichzeitig anzeigen
response_text = ""
for chunk in response:
    content = chunk.get("message", {}).get("content", "")
    print(content, end="", flush=True)
    response_text += content

# Jetzt hast du alles gesammelt
print("\n\n✅ Streaming beendet.")

Here is the extracted data in the desired format:

```
[
  {
    "Domain": "PE",
    "Subdomain": "Mathematics (PE1)",
    "Forename": "Sascha",
    "Lastname": "Albers",
    "Fullname": "Sascha Albers",
    "Type": "Member",
    "ERC-Date": "2025",
    "review_panel": "PE1"
  },
  {
    "Domain": "LS",
    "Subdomain": "Cell Biology, Development, Stem Cells and Regeneration (LS3)",
    "Forename": "Philip",
    "Lastname": "Ingham",
    "Fullname": "Philip Ingham",
    "Type": "Chair",
    "ERC-Date": "2025",
    "review_panel": "LS3"
  },
  {
    "Domain": "SH",
    "Subdomain": "Individuals, Markets and Organisations (SH1)",
    "Forename": "Bettina",
    "Lastname": "Rockenbach",
    "Fullname": "Bettina Rockenbach",
    "Type": "Chair",
    "ERC-Date": "2025",
    "review_panel": "SH1"
  },
  {
    "Domain": "PE",
    "Subdomain": "Biotechnology and Biosystems Engineering (LS9)",
    "Forename": "Nicholas",
    "Lastname": "Talbot",
    "Fullname": "Nicholas Talbot",
    "Type": "

In [26]:
response_text

'Here is the extracted data in the desired format:\n\n```\n[\n  {\n    "Domain": "PE",\n    "Subdomain": "Mathematics (PE1)",\n    "Forename": "Sascha",\n    "Lastname": "Albers",\n    "Fullname": "Sascha Albers",\n    "Type": "Member",\n    "ERC-Date": "2025",\n    "review_panel": "PE1"\n  },\n  {\n    "Domain": "LS",\n    "Subdomain": "Cell Biology, Development, Stem Cells and Regeneration (LS3)",\n    "Forename": "Philip",\n    "Lastname": "Ingham",\n    "Fullname": "Philip Ingham",\n    "Type": "Chair",\n    "ERC-Date": "2025",\n    "review_panel": "LS3"\n  },\n  {\n    "Domain": "SH",\n    "Subdomain": "Individuals, Markets and Organisations (SH1)",\n    "Forename": "Bettina",\n    "Lastname": "Rockenbach",\n    "Fullname": "Bettina Rockenbach",\n    "Type": "Chair",\n    "ERC-Date": "2025",\n    "review_panel": "SH1"\n  },\n  {\n    "Domain": "PE",\n    "Subdomain": "Biotechnology and Biosystems Engineering (LS9)",\n    "Forename": "Nicholas",\n    "Lastname": "Talbot",\n    "Ful

In [23]:
# Verarbeite die Antwort als JSON
try:
    json_output = json.loads(response_text)
    print(json_output)
except Exception as e:
    print("Fehler beim Parsen der JSON-Antwort:", e)
    print(response_text)


Fehler beim Parsen der JSON-Antwort: Expecting value: line 1 column 1 (char 0)
Here is the extracted data in the desired format:

[
  {
    "Domain": "PE",
    "Subdomain": "Mathematics (PE1)",
    "Forename": "Sascha",
    "Lastname": "Albers",
    "Fullname": "Sascha Albers",
    "Type": "Member",
    "ERC-Date": "2024",
    "review_panel": "PE1"
  },
  {
    "Domain": "LS",
    "Subdomain": "Biotechnology and Biosystems Engineering (LS9)",
    "Forename": "Nicholas",
    "Lastname": "Talbot",
    "Fullname": "Nicholas Talbot",
    "Type": "Chair",
    "ERC-Date": "2024",
    "review_panel": "LS9"
  },
  {
    "Domain": "SH",
    "Subdomain": "Texts and Concept (SH5)",
    "Forename": "Sonja",
    "Lastname": "Smets",
    "Fullname": "Sonja Smets",
    "Type": "Chair",
    "ERC-Date": "2024",
    "review_panel": "SH5"
  },
  {
    "Domain": "PE",
    "Subdomain": "Individuals, Markets and Organisations (SH1)",
    "Forename": "Bettina",
    "Lastname": "Rockenbach",
    "Fullname": "

In [24]:
# 🔍 JSON-Teil extrahieren
json_match = re.search(r'\[\s*{.*}\s*\]', response_text, re.DOTALL)
if json_match:
    json_text = json_match.group(0)
    data = json.loads(json_text)
    df = pd.DataFrame(data)
else:
    raise ValueError("Kein JSON im Text gefunden")

display(df)

,Domain,Subdomain,Forename,Lastname,Fullname,Type,ERC-Date,review_panel
0,PE,Mathematics (PE1),Sascha,Albers,Sascha Albers,Member,2024,PE1
1,LS,Biotechnology and Biosystems Engineering (LS9),Nicholas,Talbot,Nicholas Talbot,Chair,2024,LS9
2,SH,Texts and Concept (SH5),Sonja,Smets,Sonja Smets,Chair,2024,SH5
3,PE,"Individuals, Markets and Organisations (SH1)",Bettina,Rockenbach,Bettina Rockenbach,Chair,2024,SH1
4,LS,Neuroscience and Disorders of the Nervous Syst...,Christian,Büchel,Christian Büchel,Chair,2024,LS5
5,SH,The Social World and Its Interactions (SH3),Olivier,Servais,Olivier Servais,Chair,2024,SH3
6,PE,"Cell Biology, Development, Stem Cells and Rege...",Philip,Ingham,Philip Ingham,Chair,2024,LS3
7,SH,"Human Mobility, Environment, and Space (SH7)",Ole,Mertz,Ole Mertz,Chair,2024,SH7


In [11]:
response = ollama.chat(
    model="llama3:instruct",
    messages=[{"role": "user", "content": "Schreibe eine Kurze Beschreibung über die Researchfelder eines ERC Panel Mitglieds. Die Researchfelder sind: Birds, Biology, Ecology, Evolution"}]
)
print(response["message"]["content"])

KeyboardInterrupt: 